In [118]:
!pip install pyspark
!pip install py4j
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
from google.colab import files
movies = files.upload()

Saving movies.csv to movies (1).csv


In [120]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local") \
.appName("trial")  \
.config('spark.ui.port','4050') \
.getOrCreate()

In [121]:
spark

In [122]:
dataframe = spark.read.csv('movies.csv')
dataframe.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [123]:
dataframe = spark.read.format("csv").option("header","false").load("movies.csv")
dataframe.show()

+--------------------+--------------------+----+
|                 _c0|                 _c1| _c2|
+--------------------+--------------------+----+
|1::Toy Story (199...|                null|null|
|2::Jumanji (1995)...|                null|null|
|3::Grumpier Old M...|                null|null|
|4::Waiting to Exh...|                null|null|
|5::Father of the ...|                null|null|
|6::Heat (1995)::A...|                null|null|
|7::Sabrina (1995)...|                null|null|
|8::Tom and Huck (...|                null|null|
|9::Sudden Death (...|                null|null|
|10::GoldenEye (19...|                null|null|
|11::American Pres...| The (1995)::Come...|null|
|12::Dracula: Dead...|                null|null|
|13::Balto (1995):...|                null|null|
|14::Nixon (1995):...|                null|null|
|15::Cutthroat Isl...|                null|null|
|16::Casino (1995)...|                null|null|
|17::Sense and Sen...|                null|null|
|18::Four Rooms (1..

In [124]:
from pyspark.context import SparkContext
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [125]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col 
import pyspark.sql.functions as f

In [126]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [160]:
schema = StructType([
    StructField("Id",IntegerType(),True), \
    StructField("movie_name",StringType(),True), \
    StructField("movies genre",StringType(),True),\
                     ])

In [161]:
df=spark.read.csv("movies.csv", sep="::",schema=schema)
df.show(truncate = False)

+---+-------------------------------------+------------------------------+
|Id |movie_name                           |movies genre                  |
+---+-------------------------------------+------------------------------+
|1  |Toy Story (1995)                     |Animation|Children's|Comedy,, |
|2  |Jumanji (1995)                       |Adventure|Children's|Fantasy,,|
|3  |Grumpier Old Men (1995)              |Comedy|Romance,,              |
|4  |Waiting to Exhale (1995)             |Comedy|Drama,,                |
|5  |Father of the Bride Part II (1995)   |Comedy,,                      |
|6  |Heat (1995)                          |Action|Crime|Thriller,,       |
|7  |Sabrina (1995)                       |Comedy|Romance,,              |
|8  |Tom and Huck (1995)                  |Adventure|Children's,,        |
|9  |Sudden Death (1995)                  |Action,,                      |
|10 |GoldenEye (1995)                     |Action|Adventure|Thriller,,   |
|11 |American President, 

In [163]:
df3 = df.withColumn('year2', split(df['movie_name'], " ").getItem(1)).show(5)

+---+--------------------+--------------------+------+
| Id|          movie_name|        movies genre| year2|
+---+--------------------+--------------------+------+
|  1|    Toy Story (1995)|Animation|Childre...| Story|
|  2|      Jumanji (1995)|Adventure|Childre...|(1995)|
|  3|Grumpier Old Men ...|    Comedy|Romance,,|   Old|
|  4|Waiting to Exhale...|      Comedy|Drama,,|    to|
|  5|Father of the Bri...|            Comedy,,|    of|
+---+--------------------+--------------------+------+
only showing top 5 rows



In [164]:
df4 = df.withColumn("Year_Release", f.regexp_extract(df['movie_name'], '[1-9][0-9]*',0))
df4.show(5)


+---+--------------------+--------------------+------------+
| Id|          movie_name|        movies genre|Year_Release|
+---+--------------------+--------------------+------------+
|  1|    Toy Story (1995)|Animation|Childre...|        1995|
|  2|      Jumanji (1995)|Adventure|Childre...|        1995|
|  3|Grumpier Old Men ...|    Comedy|Romance,,|        1995|
|  4|Waiting to Exhale...|      Comedy|Drama,,|        1995|
|  5|Father of the Bri...|            Comedy,,|        1995|
+---+--------------------+--------------------+------------+
only showing top 5 rows



In [165]:
df4.show()

+---+--------------------+--------------------+------------+
| Id|          movie_name|        movies genre|Year_Release|
+---+--------------------+--------------------+------------+
|  1|    Toy Story (1995)|Animation|Childre...|        1995|
|  2|      Jumanji (1995)|Adventure|Childre...|        1995|
|  3|Grumpier Old Men ...|    Comedy|Romance,,|        1995|
|  4|Waiting to Exhale...|      Comedy|Drama,,|        1995|
|  5|Father of the Bri...|            Comedy,,|        1995|
|  6|         Heat (1995)|Action|Crime|Thri...|        1995|
|  7|      Sabrina (1995)|    Comedy|Romance,,|        1995|
|  8| Tom and Huck (1995)|Adventure|Childre...|        1995|
|  9| Sudden Death (1995)|            Action,,|        1995|
| 10|    GoldenEye (1995)|Action|Adventure|...|        1995|
| 11|American Presiden...|Comedy|Drama|Roma...|        1995|
| 12|Dracula: Dead and...|     Comedy|Horror,,|        1995|
| 13|        Balto (1995)|Animation|Childre...|        1995|
| 14|        Nixon (1995

In [166]:
from pyspark.sql.functions import min, max, mean
df4.select(max("Year_Release")).show()
df4.select(min("Year_Release")).show()


+-----------------+
|max(Year_Release)|
+-----------------+
|               98|
+-----------------+

+-----------------+
|min(Year_Release)|
+-----------------+
|                1|
+-----------------+



+-----------------+
|max(Year_Release)|
+-----------------+
|               98|
+-----------------+

+-----------------+
|min(Year_Release)|
+-----------------+
|                1|
+-----------------+



In [147]:
from google.colab import files
users1 = files.upload()

Saving users.csv to users (3).csv


In [148]:
schema1 = StructType([
    StructField("Id",IntegerType(),True), \
    StructField("gender",StringType(),True), \
    StructField("movieID",IntegerType(),True),\
    StructField("rating",IntegerType(),True), \
    StructField("numbers",IntegerType(),True)\
                     ])

In [154]:
dfz=spark.read.csv("users.csv", sep="::",schema=schema1)
dfz.show()

+----+------+-------+------+-------+
|  Id|gender|movieID|rating|numbers|
+----+------+-------+------+-------+
|2022|  null|   null|  null|   null|
|null|  null|   null|  null|   null|
|null|  null|   null|  null|   null|
|   1|     F|      1|    10|  48067|
|   2|     M|     56|    16|  70072|
|   3|     M|     25|    15|  55117|
|   4|     M|     45|     7|   2460|
|   5|     M|     25|    20|  55455|
|   6|     F|     50|     9|  55117|
|   7|     M|     35|     1|   6810|
|   8|     M|     25|    12|  11413|
|   9|     M|     25|    17|  61614|
|  10|     F|     35|     1|  95370|
|  11|     F|     25|     1|   4093|
|  12|     M|     25|    12|  32793|
|  13|     M|     45|     1|  93304|
|  14|     M|     35|     0|  60126|
|  15|     M|     25|     7|  22903|
|  16|     F|     35|     0|  20670|
|  17|     M|     50|     1|  95350|
+----+------+-------+------+-------+
only showing top 20 rows



In [155]:
dfz.select(max('gender'))

DataFrame[max(gender): string]

In [158]:
dfz.drop("gender")

DataFrame[Id: int, movieID: int, rating: int, numbers: int]